<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/11_cnn_pytorch/02_5_FT_and_TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ネットワークアーキテクチャ
 ネットワークの構造はあらかじめ人が決めておく必要があります．専門家でなければ，どのようなネットワーク構造が良いのか，１から決めるのは至難の技です．そこで，コンペティションなどで高い精度を達成しているネットワーク構造を利用します．
 

## AlexNet
AlexNetは，2012年のILSVRCで優勝したモデルであり，畳み込み層が5層，全結合層が３層の８層構造です．当時のGPUのメモリ制約から，学習時には，各層の特徴マップをチャネル方向に分割し，2台のGPUで独立して学習するというアプローチが取られています．CNNの重みはガウス分布に従う乱数により初期化し，モーメンタム付きの確率的勾配降下法 (Stochastic Gradient Descent; SGD) により最適化します．学習時，誤差が低下しなくなったタイミングで学習率を1/10に減少させることも行われており，この最適化手法は，現在においてもベストプラクティスとして利用されています．AlexNetに導入された重要な要素技術として，Rectified Linear Units (ReLU)，Local Response Normalization (LRN)，Overlapping Pooling，Dropoutです．ReLUは，深いネットワークにおいてシグモイド関数などの従来の活性化関数を利用した場合に発生する勾配消失問題を解決することができる活性化関数です．LRNは，特徴マップの同一の位置にあり，隣接するチャネルの出力の値から，自身の出力の値を正規化する手法です．プーリング層は，sピクセルずつ離れたグリッドにおいて，周辺zピクセルの最大値や平均値で集約する処理を行います．通常はs = zとして重ならないようにしています．一方で，AlexNetでは，s = 2， z = 3として，集約されるピクセル領域がオーバラップするようにしています．このoverlapping poolingにより，過学習を低減し，わずかに最終的な精度が向上します．Dropoutは，学習時のネットワークについて，全結合層のユニットを一定確率で無効化する手法です．これにより，擬似的に毎回異なるアーキテクチャで学習を行うこととなり，アンサンブル学習と同様の効果が得られるため，より汎化されたモデルを学習することができます．

<img src="https://github.com/himidev/Lecture/blob/main/11_cnn_pytorch/02_5_finetuning/AlexNet.png?raw=true" width = 90%>


## VGGNet
VGGNetは，2014年のILSVRCにおいて，2位の認識精度を達成したモデルです．VGGNetは，シンプルな構造をしており，また学習済みモデルが公開されたことから，現在においてもベースラインのモデルとして利用されています．VGGNetは，1)フィルタサイズを3×3に統一する，2)同一フィルタ数の畳み込み層を幾つか積層した後に最大値プーリングを行い，特徴マップを半分に縮小する，3)最大値プーリング後の畳み込み層のフィルタ数を2倍に増加させる，というアイデアを導入しています． VGGNetには，いくつかのバリエーションがありますが，代表的な構造は13層の畳み込み層と3層の全結合層の全16層の構成です．VGGNetは，従来と比較して深いネットワークのため学習が難しいです．そのため，まず浅いネットワークを学習し，その後畳み込み層を追加した深いネットワークを学習する方法を取り入れています．


<img src="https://github.com/himidev/Lecture/blob/main/11_cnn_pytorch/02_5_finetuning/VGG.png?raw=true" width = 70%>


## GoogLeNet
GoogLeNetは，2014年のILSVRCの優勝モデルであり，Inception モジュールという複数の畳み込み層やpooling層から構成される小さなネットワークを9つ積層して全22層のネットワーク構造です．Inceptionモジュールは，ネットワークを分岐させてサイズの異なる畳み込みを行った後，それらの出力を連結する処理を行っています．また，GoogLeNetは，Global Average Pooling (GAP)を導入しています．従来のCNNモデルは，畳み込み層の後に複数の全結合層を重ねることで，最終的なクラス分類の出力を得る構造となっています．全結合層はパラメータ数が多く，また過学習を起こすことが課題となっており，dropoutを導入することで過学習を抑える必要がありました．一方，GAPは，入力された特徴マップのサイズと同じサイズのaverage poolingを行うプーリング層です．CNNの最後の畳み込み層の出力チャネル数を最終的な出力の次元数（クラス数）と同一とし，その後にGAPを適用することで，全結合層を利用することなく最終的な出力を得ることを提案しています．全結合層を利用しないことで，パラメータ数を大きく削減し，過学習を防ぐことができるようになりました．

<img src="https://github.com/himidev/Lecture/blob/main/11_cnn_pytorch/02_5_finetuning/GoogleNet.png?raw=true" width = 100%>

## ResNet
Residual Networks (ResNet)は，2015年のILSVRCの優勝モデルです．VGGNetで示されたように，ネットワークを深くすることは表現能力を向上させ，認識精度を改善できます．しかし，あまりにも深いネットワークは効率的な学習が困難でした．ResNetは，通常のネットワークのように，何かしらの処理ブロックによる変換F(x)を単純に次の層に渡していくのではなく，その処理ブロックへの入力xをショートカットし， H(x) = F(x)+xを次の層に渡すようにしています．このショートカットを含めた処理単位をResidual Blockと呼びます．ResNetでは，ショートカットを通して，誤差逆伝播法時に勾配が直接下層に伝わっていくことになり，非常に深いネットワークにおいても効率的に学習ができるようになりました．Residual Blockは，3×3 のフィルタサイズを持つ畳み込み層とバッチ正規化，ReLUから構成されています．深いネットワークでは，ある層のパラメータの更新によって，その次の層への入力の分布がバッチ毎に大きく変化してまう内部共変量シフト (internal covariate shift) が発生し，学習が効率的に進まない問題がありました．バッチ正規化は，内部共変量シフトを正規化し，なるべく各レイヤが独立して学習が行えるようにすることで，学習を安定化・高速化する手法です．ResNetではこのバッチ正規化とショートカットをResidualモジュールに組み込むことで152層と非常に深いネットワークの学習を実現しています．

ResNetが登場後，様々な派生系が提案され，代表的なネットワーク構造としては，WideResNet, PyramidNet, ResNeXt，Xception，DenseNetなどがあります．

<img src="https://github.com/himidev/Lecture/blob/main/11_cnn_pytorch/02_5_finetuning/ResidualBlock.png?raw=true" width = 70%>

<img src="https://github.com/himidev/Lecture/blob/main/11_cnn_pytorch/02_5_finetuning/ResNet.png?raw=true" width = 100%>

## アーキテクチャの自動探索
人が最適なネットワーク構造を探し出すことはほぼ不可能です．そのため，学習により準最適なネットワーク構造の探索を行うことが注目されています．ネットワーク構造には，層数だけでなく，フィルタサイズやフィルタ数，また特徴マップの連結方法など，決めるべき対象が多く，全ての組み合わせを含めた最適化はコンピュータを利用したとしてもほぼ不可能です．そのため，ある程度設計したブロックをどのように組み合わせたら良いかを学習する手法が主流となっています．代表的な手法であるNeural Architecture Search (NAS) では，強化学習でネットワーク構造を探索しています．NASNetでは，探索する対象をResNetのResidual Blockのような塊（セル）をどのように組み合わせるかに焦点を当てています．MnasNetでは，モバイル端末などでも動作するようなネットワーク構造を探索します．NASやNASNetでは，認識精度を強化学習の報酬にしていましたが，MnasNetでは，処理速度も報酬に加えています．これにより，精度と速度を両立させたネットワーク構造を獲得できるようになりました．

## EfficientNet
EfficientNetは，MnasNetにより得たネットワーク構造を拡張して精度を重視するネットワークに拡張しています．この時，各層のフィルタ数や層数，入力画像サイズの関係を定式化し，１つのパラメータでこれらの値を決定します．これにより，非常に簡単に新たな高精度なネットワーク構造を獲得できるようになりました．EfficientNetは，高精度度なだけでなく，様々なタスクのベースネットワークに活用され，転移学習に有用なネットワーク構造となっています．

<img src="https://github.com/himidev/Lecture/blob/main/11_cnn_pytorch/02_5_finetuning/EfficientNet.png?raw=true" width = 100%>

<img src="https://github.com/himidev/Lecture/blob/main/11_cnn_pytorch/02_5_finetuning/EfficientNetPerformance.png?raw=true" width = 60%>



## 準備
### Google Colaboratoryの設定確認・変更
本チュートリアルではPyTorchを利用してニューラルネットワークの実装を確認，学習および評価を行います．
**GPUを用いて処理を行うために，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．**


# ファインチューニングと転移学習

深層学習の学習では，人が設計したネットワークモデルの初期パラメータを乱数で決めて学習を行います．
別の手段として，大規模なデータセットで学習したパラメータを初期パラメータとして，学習を行う方法もあります．
この時，学習には別の（小規模な）データセットを利用します．
すなわち，小規模なデータセットで高い精度を達成するために，あらかじめ汎用的な特徴を大規模なデータセットで獲得しているネットワークを活用することになります．
これを転移学習と言います．
大規模なデータセットには，ImageNet(学習データ数100万枚)を利用することが多いです．
深層学習の場合，利用するネットワークの出力層のみを学習することを転移学習，ネットワーク全体を学習することをファインチューニングと呼びます．ここでは，まず全体を学習するファインチューニングから説明します．

##ファインチューニング
まず，学習を行う小規模なデータセットをダウンロードします．ここでは，Pytorchのチュートリアルでも利用しているハチとアリの２クラス識別を行うデータセットを利用します．

In [ ]:
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip 
!unzip -q hymenoptera_data.zip

--2022-05-05 12:24:03--  https://download.pytorch.org/tutorial/hymenoptera_data.zip
Resolving download.pytorch.org (download.pytorch.org)... 52.84.225.58, 52.84.225.92, 52.84.225.122, ...
Connecting to download.pytorch.org (download.pytorch.org)|52.84.225.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47286322 (45M) [application/zip]
Saving to: ‘hymenoptera_data.zip’

hymenoptera_data.zi 100%[===================>]  45.10M  69.7MB/s    in 0.6s    

2022-05-05 12:24:04 (69.7 MB/s) - ‘hymenoptera_data.zip’ saved [47286322/47286322]



hymenoptera_dataというディレクトリの中にtrainとvalディレクトリがあります．その中にantsとbeesというディレクトリがあります．このようにクラスごとにデータを用意しておきます．


### モジュールのインポート
はじめに必要なモジュールをインポートします．

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU availability: True`と表示されれば，GPUを使用した計算をPyTorchで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．

In [ ]:
# モジュールのインポート
from time import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import torchvision
from torchvision import datasets, models, transforms

import torchsummary

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

### データセットの読み込みと確認

データ拡張の設定を行います．ここでは，ランダムリサイズクロップとランダムクリップをします．また，画像の正規化(明るさの正規化)も行います．
データセットおよびデータ拡張をデータローダに与えます．
hymenoptera_data/trainのデータをtrain_data, hymenoptera_data/valのデータをval_dataとします．datasets.ImageFolderを利用すると，指定したディレクトリ内にある各サブデイレクトリをクラスに対応してくれます．
そして，クラス名はディレクトリ名から作成して学習データの場合はtrain_data.classesに含まれています．

In [ ]:
train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
test_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


data_dir = 'hymenoptera_data'
train_data = datasets.ImageFolder("hymenoptera_data/train", train_transforms)
val_data = datasets.ImageFolder("hymenoptera_data/val", test_transforms)

class_names = train_data.classes
print(class_names)

学習済みモデルにはResNet18を利用します．pretrained = Trueにすると，ImageNetで学習したモデルを利用できます．ここで，ImageNetは1000クラスのデータセットです．すなわち，ImageNetで学習したResNet18の出力層のユニット数は1000になっています．ファインチューニングに利用するデータセットはハチとアリの２クラスなので，出力層のユニット数を変更します．

In [ ]:
model = models.resnet18(pretrained=True)
print("======== Original netowrk architecutre ========\n")
print(model)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
print("======== Fine-funing netowrk architecutre ========\n")
print(model)

if use_cuda:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# モデルの情報を表示
torchsummary.summary(model, (3, 224, 224))



### 学習
１回の誤差を算出するデータ数（ミニバッチサイズ）を16，学習エポック数を25とします．
hymenoptera_dataの学習データ数を取得し，１エポック内における更新回数を求めます．
学習モデルに`image`を与えて各クラスの確率yを取得します．各クラスの確率yと教師ラベル`label`との誤差をsoftmax coross entropy誤差関数で算出します．
また，認識精度も算出します．そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．


In [ ]:
# ミニバッチサイズ・エポック数の設定
batch_size = 16
epoch_num = 25
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model.train()


start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        if use_cuda:
            image = image.cuda()
            label = label.cuda()

        y = model(image)
        loss = criterion(y, label)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_data),
                                                                                 time() - start))


### テスト
学習したネットワークモデルを用いて評価を行います．

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(val_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / len(val_data)))

認識結果を確認します．

In [ ]:
def tensor_to_numpy(inp):
  "imshow for Tensor"
  inp = inp.numpy().transpose((1,2,0))
  mean = np.array([0.485, 0.456, 0.406])
  std = np.array([0.229, 0.224, 0.225])
  inp = std * inp + mean
  inp = np.clip(inp, 0, 1)
  return inp

model.eval()
num_images = 4 
count = 0
fig = plt.figure()

# データローダーの準備
test_loader = torch.utils.data.DataLoader(val_data, batch_size=num_images, shuffle=True)

with torch.no_grad():
    for i, (image, label) in enumerate(test_loader):
        image = image.cuda()
        label = label.cuda()

        y = model(image)
        _, preds = torch.max(y, 1)

        for j in range(image.size()[0]):
            count += 1
            ax = fig.add_subplot(num_images//2, 2, count)
            ax.axis('off')
            ax.set_title('predicted: {}  label: {}'
                         .format(class_names[preds[j]], class_names[label[j]]))
            ax.imshow(tensor_to_numpy(image.cpu().data[j]))

        if count >= num_images:
            break
    



###Grad-CAMによるAttention mapの可視化
Grad-CAMを利用するために必要なツールをインストールします．
Grad-CAMは，`pytorch-gradcam`というツールをインストールすることで簡単に利用することができます．


In [ ]:
!pip install pytorch-gradcam

Grad-CAMによりAttention mapを可視化して，ネットワークの判断根拠を確認してみます． ここでは，hymenoptera_dataのフォルダにある画像を直接指定して可視化させます．可視化には，には，Pytorchのtorchvisionに用意されている用意されているmake_gridを利用します．

In [ ]:
from gradcam.utils import visualize_cam
from gradcam import GradCAM
from torchvision.utils import make_grid
from PIL import Image

# Grad-CAM
target_layer = model.layer4 # ex., layer1, layer2, layer3, layer3[1], layer4[0]
gradcam = GradCAM(model, target_layer)

img = Image.open("/content/hymenoptera_data/val/ants/1053149811_f62a3410d3.jpg")

torch_img = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])(img)

if use_cuda:
    torch_img = torch_img.cuda()

normed_input_img = transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])(torch_img)[None]
mask, _ = gradcam(normed_input_img)
heatmap, result = visualize_cam(mask, torch_img)

grid_image = make_grid([torch_img.cpu(), result], nrow=5)

transforms.ToPILImage()(grid_image)




##転移学習
ネットワークの変更した層のみを学習させて転移学習します．
その場合，各層に勾配を逆伝播しないよう，requires_grad を Falseにします．

In [ ]:
model = models.resnet18(pretrained=True)
print("======== Original netowrk architecutre ========\n")
print(model)

for param in model.parameters():
    param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
print("======== Fine-funing netowrk architecutre ========\n")
print(model)

if use_cuda:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# モデルの情報を表示
torchsummary.summary(model, (3, 224, 224))

### 学習
１回の誤差を算出するデータ数（ミニバッチサイズ）を16，学習エポック数を25とします．
hymenoptera_dataの学習データ数を取得し，１エポック内における更新回数を求めます．
学習モデルに`image`を与えて各クラスの確率yを取得します．各クラスの確率yと教師ラベル`label`との誤差をsoftmax coross entropy誤差関数で算出します．
また，認識精度も算出します．そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．


In [ ]:
# ミニバッチサイズ・エポック数の設定
batch_size = 16
epoch_num = 25
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model.train()


start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        if use_cuda:
            image = image.cuda()
            label = label.cuda()

        y = model(image)
        loss = criterion(y, label)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_data),
                                                                                 time() - start))


### テスト
学習したネットワークモデルを用いて評価を行います．

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(val_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / len(val_data)))

##EfficientNetのファインチューニング
ResNetとともに高い精度を達成しているネットワークモデルであるEfficientNetを学習済みモデルとして利用します．EfficientNetをpipでインストールします．EfficientNetはB0からB7までネットワーク構造が違うモデルがあります．ここではB1を利用します．学習済みモデルを読み込む際，転移学習後のクラス数を指定することができます．

In [ ]:
!pip install efficientnet_pytorch


In [ ]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b1', num_classes = 2) 
print("======== Fine-funing netowrk architecutre ========\n")
print(model)

if use_cuda:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# モデルの情報を表示
torchsummary.summary(model, (3, 224, 224))

### 学習
１回の誤差を算出するデータ数（ミニバッチサイズ）を16，学習エポック数を25とします．
hymenoptera_dataの学習データ数を取得し，１エポック内における更新回数を求めます．
学習モデルに`image`を与えて各クラスの確率yを取得します．各クラスの確率yと教師ラベル`label`との誤差をsoftmax coross entropy誤差関数で算出します．
また，認識精度も算出します．そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．


In [ ]:
# ミニバッチサイズ・エポック数の設定
batch_size = 16
epoch_num = 25
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model.train()


start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        if use_cuda:
            image = image.cuda()
            label = label.cuda()

        y = model(image)
        loss = criterion(y, label)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_data),
                                                                                 time() - start))


### テスト
学習したネットワークモデルを用いて評価を行います．

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(val_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / len(val_data)))

#課題
1. EfficientNetをB1からそれ以外の構造に変えて精度の変化を比較しましょう．
 "efficientnet-b1" を　"efficientnet-b0"，"efficientnet-b2"，，，"efficientnet-b7"のいずれかにすれば良いです．

In [ ]:
#ここにコードを書く

2. AlexNetで学習済みモデルを利用する場合としない場合での精度を比較しましょう．

In [ ]:
#ヒント: AlexNet(学習済みモデル)を２クラス識別にするためには以下のようにします．
model = models.alexnet(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 2)